## Installing

Installing [PyTorch](http://pytorch.org) is fairly easy:  
`pip install torch torchvision` or `conda install torch torchvision`

Note:  
- This tutorial is written for Python3.6+, which I strongly recommend using. It has lots of wholesome features!
- This file is a [Jupyter Notebook](https://jupyter.org/) `pip install jupyter; jupyter notebook`, which is a very handy way of presenting python experiments.

## What is PyTorch?

PyTorch...
- is a numerical computation library,
- has both CPU and GPU backends,
- has a seamless automatic differentiation package, think gradients, $\nabla_\theta f$
- is finetuned for Deep Learning, but can be used for anything!

In [1]:
import torch
import numpy as np

The fundamental data structure in PyTorch is the `Tensor`, which is a multidimensional array.

In [2]:
a = torch.tensor([1,2,3])
b = torch.tensor([4,5,6])
print('a + b =', a + b)   # These 3 operations are `elementwise`
print('a - b =', a - b)
print('a * b =', a * b)
print('a^T b =', a.dot(b))

a + b = tensor([5, 7, 9])
a - b = tensor([-3, -3, -3])
a * b = tensor([ 4, 10, 18])
a^T b = tensor(32)


Tensors have a shape, and thus a certain number of dimensions, that can both be dynamically changed. Keep in mind when reshaping that the array data is row-major.

In [3]:
c = torch.eye(4, 4)
print(c)
print(c.reshape((2, 8)))
d = torch.arange(9)
print()
print(d)
print(d.reshape((3, 3)))

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
tensor([[1., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 1.]])

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])


Arrays can be indexed and sliced in various ways

In [4]:
d = d.reshape((3, 3))
print('indexing: ')
print(d[1, 1])
print(d[:2])
print(d[:, :2])
print(d[1:3, 1:])

indexing: 
tensor(4)
tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[0, 1],
        [3, 4],
        [6, 7]])
tensor([[4, 5],
        [7, 8]])


## Broadcasting

PyTorch's tensor elementwise operations follow *broadcasting* rules when the shapes differ:
> Two tensors are “broadcastable” if the following rules hold:
> - Each tensor has at least one dimension.
> - When iterating over the dimension sizes, starting at the trailing dimension, the dimension sizes must either be equal, one of them is 1, or one of them does not exist.

What does this mean?
For example:

In [5]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([10])
print('a and b\'s shapes:', a.shape, b.shape)
# Since b.shape[0] == 1, `10` is "broadcast", i.e. virtually copied, across a's 0th dimension
# So [1, 2, 3] + [10] is equivalent to [1, 2, 3] + [10, 10, 10]
print('a + b:', a + b)
print('a + b shape:',(a + b).shape)

a and b's shapes: torch.Size([3]) torch.Size([1])
a + b: tensor([11, 12, 13])
a + b shape: torch.Size([3])


In [6]:
# More complicated:
# - since the second tensor only has 3 dimensions, a dimension of length `1` is implicitly added.
# - the 0th and 2nd dimensions are now of length `1` in the second tensor, so they are broadcasted!
print((torch.ones(5,4,3,2) + 
       torch.ones(  4,1,2)).shape)

# This does not work! `1`-dimensions can only be added to the left 
try:
    print((torch.ones(5,4,3,2) + 
           torch.ones(5,4,3  )).shape)
except:
    print("That did not work, as expected.")
    
# scalars are implicitly broadcast to every dimension:
print(torch.eye(3,3) * np.pi)

torch.Size([5, 4, 3, 2])
That did not work, as expected.
tensor([[3.1416, 0.0000, 0.0000],
        [0.0000, 3.1416, 0.0000],
        [0.0000, 0.0000, 3.1416]])


### Advanced indexing

In [7]:
# It is possible to index a dimension along specified indices
array = torch.arange(7*3).reshape((7,3))
print(array)
# This is equivalent to [array[1], array[3], array[5]]
print(array[[1, 3, 5]])
# Indexing over multiple dimensions is also allowed:
# This is equivalent to [array[1, 0], array[3, 1], array[5, 0]]
print(array[[1, 3, 5], [0, 1, 0]])

print(array[:, [0, 2]])

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17],
        [18, 19, 20]])
tensor([[ 3,  4,  5],
        [ 9, 10, 11],
        [15, 16, 17]])
tensor([ 3, 10, 15])
tensor([[ 0,  2],
        [ 3,  5],
        [ 6,  8],
        [ 9, 11],
        [12, 14],
        [15, 17],
        [18, 20]])


In [8]:
# `None` can be used to add dimesions
print('d[None, :2, None, 1:].shape = ', d[None, :2, None, 1:].shape) 

# These dimensions become of length 1, which is broadcastable!
# For example a matrix multiply is:
A = (d[:, None, :] * d.transpose(0, 1)[:, :, None]).sum(dim=0)
# but this is more efficient:
B = torch.mm(d, d)
# and this is more elegant, if you're into that:
C = torch.einsum('ij,jk->ik', d, d)

print('\nDoes A == B == C?',
      bool((A == B).all() and (B == C).all())) # Checking that `all` elements are equal
print(A)

d[None, :2, None, 1:].shape =  torch.Size([1, 2, 1, 2])

Does A == B == C? True
tensor([[ 15,  18,  21],
        [ 42,  54,  66],
        [ 69,  90, 111]])


### Parenthesis on einsum
Parenthesis on **einsum**, tensor product-sums in Einstein summation notation:

`C = einsum('ijk,kji->j', A, B)` is equivalent to (`A` and `B` are 3D Tensors)
$$C_j = \sum_i\sum_k A_{ijk} B_{kji}$$

`C = einsum('ij,k->ijk', A, B)` is equivalent to (`A` a matrix, `B` a vector)
$$C_{ijk} = A_{ij} B_k$$
`C = einsum('ij,k->ij', A, B)` is equivalent to (idem)
$$C_{ij} = \sum_k A_{ij} B_k$$

Whenever an index is used for two or more tensors (e.g. `i` in the first example), the shape **has** to match along the given dimensions. 

Even if the dimension shapes match, you don't have to use the same letter, e.g.:

`C = einsum('ij,kl->', A, B)` is equivalent to (`A` and `B` matrices)
$$C = \sum_i \sum_j\sum_k\sum_l A_{ij}B_{kl}$$
note the lack of index after `->`, which makes $C\in\mathbb{R}$ a scalar.

## Automatic Differentiation

This is the main feature of PyTorch! It makes gradient-based optimization extremely easy to implement.

All tensors have the potential to store gradient information, but they must be set to:

In [9]:
x = torch.tensor([[1, 0.4, 3, 0.6]], dtype=torch.float, requires_grad=True)
W = torch.tensor(np.random.uniform(-1, 1, (4, 4)), dtype=torch.float, requires_grad=True)

Now whenever we compute things with `x` and `W`, PyTorch keeps track of the operations in a computation graph.

In [10]:
h = torch.tanh(x.mm(W))
u = torch.sum(h ** 2)
print(u)

tensor(3.5122, grad_fn=<SumBackward0>)


Now the we have a scalar "loss" `u`, calling `u.backward()` will fill `x` and `W`'s `grad` attribute with $\nabla_xu$ and $\nabla_Wu$ respectively:

In [11]:
u.backward()
print(x.grad)
print(W.grad)

tensor([[ 0.1653, -0.4894,  0.3756,  0.5888]])
tensor([[ 0.0306, -0.2640, -0.3043, -0.2985],
        [ 0.0122, -0.1056, -0.1217, -0.1194],
        [ 0.0918, -0.7920, -0.9128, -0.8955],
        [ 0.0184, -0.1584, -0.1826, -0.1791]])


Careful, this is additive! Calling backward again with a different loss will *add* to `grad`.

In [12]:
h = torch.tanh(x.mm(W))
v = torch.sum(torch.exp(-h / 2))
v.backward()
print(x.grad)
print(W.grad)

tensor([[ 0.2250, -0.7066,  0.5252,  0.8447]])
tensor([[ 0.0259, -0.3772, -0.4358, -0.4274],
        [ 0.0104, -0.1509, -0.1743, -0.1710],
        [ 0.0777, -1.1317, -1.3074, -1.2821],
        [ 0.0155, -0.2263, -0.2615, -0.2564]])


But gradients can be zeroed out:

In [13]:
x.grad.zero_() # Side note, all torch functions ending in "_" mean that they change something in-place.
W.grad.zero_()

# Let's do it again and check that it is different
h = torch.tanh(x.mm(W))
v = torch.sum(torch.exp(-h / 2))
v.backward()
print(x.grad)
print(W.grad)

tensor([[ 0.0596, -0.2172,  0.1496,  0.2559]])
tensor([[-0.0047, -0.1132, -0.1315, -0.1289],
        [-0.0019, -0.0453, -0.0526, -0.0516],
        [-0.0141, -0.3397, -0.3946, -0.3866],
        [-0.0028, -0.0679, -0.0789, -0.0773]])


## Optimizers

Now that we can compute gradients, we can try to run optimization methods.

Let's start with just gradient descent, for that we will need some data:

In [14]:
import urllib.request
from io import StringIO
# Fetch the data
iris = urllib.request.urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data").read()
# Replace string labels by class numbers
iris = (iris
        .decode('ascii')
        .replace('Iris-setosa', '0')
        .replace('Iris-versicolor', '1')
        .replace('Iris-virginica', '2'))
# Parse into a numpy array
iris = np.loadtxt(StringIO(iris), delimiter=',')
print(iris.shape)
print(iris[:3])

(150, 5)
[[5.1 3.5 1.4 0.2 0. ]
 [4.9 3.  1.4 0.2 0. ]
 [4.7 3.2 1.3 0.2 0. ]]


In [15]:
np.random.seed(1)  # Just to keep things constant from run to run
torch.random.manual_seed(1)
# Let's make a training set and a test set:
indices = np.arange(len(iris))
np.random.shuffle(indices)
train_split, test_split = indices[:100], indices[100:]

# The last column are the Ys
trainX, trainY = iris[train_split][:, :-1], iris[train_split][:, -1]
testX, testY = iris[test_split][:, :-1], iris[test_split][:, -1]

Let's start with a simple linear classifier.

In [16]:
W = torch.nn.Parameter(torch.zeros((4, 3)).float()) # There are 4 features and 3 classes
W.data.uniform_(-0.01, 0.01) # Fill with random weights

b = torch.nn.Parameter(torch.zeros((3,)).float())

# The optimizer class takes in parameters and a learning rate,
# and sometimes, more hyper-parameters such as momentum rate.
optimizer = torch.optim.SGD([W, b], lr=0.01)
# Try playing with the learning rate to get a lower test error!


x = torch.tensor(trainX).float()
y = torch.tensor(trainY).long() # long because they are class labels
for epoch in range(1001):
    optimizer.zero_grad() # Reset the gradients
    yhat = torch.mm(x, W) + b
    loss = torch.nn.functional.cross_entropy(yhat, y)
    loss.backward()  # Compute the gradients
    optimizer.step() # Update the parameters
    if epoch and not epoch % 100:
        print(epoch, loss.item())
        
print()
train_class_predictions = yhat.argmax(1)
train_error_rate = (train_class_predictions != y).float().mean()
print(f'Our training error is {100*train_error_rate: 2.2f}%')
x = torch.tensor(testX).float()
y = torch.tensor(testY).long()
yhat = torch.mm(x, W) + b
test_class_predictions = yhat.argmax(1)
test_error_rate = (test_class_predictions != y).float().mean()
print(f'Our test error is     {100*test_error_rate: 2.2f}%')

100 0.6988282203674316
200 0.5784186720848083
300 0.5177507400512695
400 0.4786124527454376
500 0.4496915340423584
600 0.4265597462654114
700 0.4071331024169922
800 0.3902965188026428
900 0.37539300322532654
1000 0.3620048761367798

Our training error is  4.00%
Our test error is      6.00%


## Neural networks
Let's see if neural networks can perform better on this simple task. I only managed to get as low as 4% test error.

In [17]:
import torch.nn as nn

np.random.seed(1)  # Just to keep things constant from run to run
torch.random.manual_seed(1)


model = nn.Sequential(
    nn.Linear(4, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 3))
opt = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.99), weight_decay=1e-2)

x = torch.tensor(trainX).float()
y = torch.tensor(trainY).long() # long because they are class labels
for epoch in range(5001):
    opt.zero_grad() # Reset the gradients
    # equivalent to
    # for w in model.parameters():
    #     w.grad.data.zero_()
    yhat = model(x)
    loss = torch.nn.functional.cross_entropy(yhat, y)
    loss.backward()  # Compute the gradients
    opt.step() # Update the parameters
    if epoch and not epoch % 500:
        print(epoch, loss.item())
        
print()
train_class_predictions = yhat.argmax(1)
train_error_rate = (train_class_predictions != y).float().mean()
print(f'Our training error is {100*train_error_rate: 2.2f}%')
x = torch.tensor(testX).float()
y = torch.tensor(testY).long()
yhat = model(x)
test_class_predictions = yhat.argmax(1)
test_error_rate = (test_class_predictions != y).float().mean()
print(f'Our test error is     {100*test_error_rate: 2.2f}%')

500 0.8079302310943604
1000 0.5909492373466492
1500 0.410470575094223
2000 0.2573563754558563
2500 0.14693450927734375
3000 0.09226757287979126
3500 0.07226601988077164
4000 0.06423167884349823
4500 0.05952752009034157
5000 0.05623064935207367

Our training error is  1.00%
Our test error is      4.00%


In [18]:
# How to sample from a random vector and send to GPU
X = np.random.uniform(0, 1,(1000, 4)) # CPU
minibatch_size = 32

device = torch.device('cuda')
# This is required to send the parameters of `model` to the GPU
model.to(device)
for i in range(100):
    indices = np.random.randint(0, X.shape[0], minibatch_size)
    minibatch_x = torch.tensor(X[indices]).float().to(device) # GPU
    prediction = model(minibatch_x)

Note that you can specify which GPU to use before running your program by launching python/jupyter as follows:  
`CUDA_VISIBLE_DEVICES=2 python code.py`  
or  
`CUDA_VISIBLE_DEVICES=2 jupyter notebook`

Note that GPUs are 0-indexed, so e.g. go 0-3 if you have 4 GPUs on your machine.

For multigpu and more check [the docs](https://pytorch.org/docs/stable/notes/cuda.html).
